# Imports

In [5]:
import numpy as np # Pour les math
import matplotlib.pyplot as plt # Pour les graphiques
from scipy.optimize import linprog # Notre solveur

# Données

In [7]:
temperatures_exterieurs = np.load("Temperatures-Montreal.npy")
print(temperatures_exterieurs[0])

-0.8299999999999841


# Résolution du système
## Variables
Soit ${e}_i$ la consomation électrique de la pompe à l'intervale $i$. \
Soit $s_i = 1$ si l'intervale $i$ se trouve le soir (entre 22H et 7H) et $s_i = 0$ sinon. \
Soit $j_i = 1$ si l'intervale $i$ se trouve la journée (entre 7H et 22H) et $j_i =0$ sinon. \
Soit $a_i = 1$ si on active la pompe à chaleur à l'intervale $i$ et $a_i = 0$ sinon. \
Soit $r_i = 1$ si la pompe à chaleur est en mode "reverse" à l'intervale $i$ et $r_i = 0$ sinon. \ 
Soit $c_i = 1$ si la pompe à chaleur est en mode "normal" à l'intervale $i$ et $c_i = 0$ sinon. \ 
Soit $t_{{in}_i}$ la température intérieure à l'intervale $i$. \
Soit $t_{{ex}_i}$ la température exterieure à l'intervale $i$.

## Objectif
Minimiser le coût total de l'électricité consommée par la pompe à chaleur. \
Étant donné qu'on sait déterminer quand nous sommes la journée ou la soirée, on obtiens ceci pour le cout qu'on vas noter $c_i = 0.18s_i + 0.26j_i$. \
En effet, étant donné que $s_i$ et $j_i$ ne peuvent pas être positifs en même temps alors il n'y aura qu'une des deux valeur choisies.

Maintenant il reste à déterminer la quantité d'énergie utilisée ($e_i$). Pour ce faire, il faut d'abord savoir si la pompe à chaleur vas chauffer ou refroidir. On doit donc implémenter ceci : 
```
si température intérieur > température extérieur alors ri = 1 
ou sinon ri = 0
```
Pour résoudre ça on vas utiliser la [Big-M method](https://en.wikipedia.org/wiki/Big_M_method)
On obtien donc les inégalitées suivantes.
$
    t_{{ex}_i} \ge t_{{in}_i} - M(1 - r_i) \\
    t_{{ex}_i} \le  t_{{in}_i} + Mr_i \\
    r_i \in \{0,1\}
$

Donc on obitens une fonction pour le COP :
$o_i = c_i(3 + 10 * abs(tanh(Text/100)) * tanh(Text/100)) + 3.2r_i$ 

Ainsi pour résoudre notre problème énoncé plus haut:
TODO : PAS SUR DE ÇA ?
TODO : Le solveur doit trouver 2 trucs, la température et quand activer la pompe à chaleur !!!!!

$
e_i = o_i * | t_{{ex}_i} - t_{{in}_i}| 
$

$\[ \sum_{i=0}^{672} a_i( (*qt de chaleure*) ) \]$

## Contraintes
Consommation électrique maximale de la pompe à chaleur: $1kW$\
${e}_i \leq 1$ \
l'intervale ne peut pas être dans la journée et dans le soir. \
$s_i = 1 - j_i$

La pompe à chaleur ne peux pas refroidir et chauffer en même temps. \
$r_i = 1 - c_i$

Les variables $ s_{i}$, $j_{i}$, $a_{i}$ sont binaires \
$s_i \in \{0, 1\}$ \
$j_i \in \{0, 1\}$ \
$a_i \in \{0, 1\}$ \
$r_i \in \{0, 1\}$ \
$c_i \in \{0, 1\}$ \

La température doit être satisfaite
$t_{{in}_i} - t_{{in}_{i-1}} = 0.1(t_{{in}_{i-1}} - t_{{ex}_i})$


$
\begin{cases}
min a \\
s_i = 1 - j_i \\
{e}_i \leq 1 \\
\end{cases}
$

In [11]:
def task_1(i_idx):
    days_offset = 96
    end_idx = i_idx + days_offset*7
    central_value = 20
    
    # On suppose qu'on commence la journée à 0
    print(central_value)

In [4]:
task_1(0)

-6.727605482178549
